# MySQL desde Python y Notebook

In [240]:
import pymysql

In [241]:
conn.close()

In [242]:
# creamos la conexión entre python y el servidor mysql
# ¡¡Recuerda arrancar el servidor con xampp!!
con = pymysql.connect('localhost','root','','provincias')

In [243]:
# creamos un cursor de acceso a queries
conn = con.cursor()

In [244]:
# enviamos la query al servidor mysql a través del cursor anterior
conn.execute("""
    SELECT * FROM provincias;
""")
# capturamos el resultado de la query desde el cursor a la tupla rs
rs = conn.fetchall()

In [245]:
import pandas as pd

In [246]:
df = pd.DataFrame(rs)
df.columns=['autonomia','provincia','poblacion','superficie']
df.head()

,autonomia,provincia,poblacion,superficie
0,Galicia,La Coruña,1122799,7950
1,Galicia,Lugo,336527,9856
2,Galicia,Orense,314853,7273
3,Galicia,Pontevedra,944346,4494
4,Principado de Asturias,Asturias,1042608,10603


### Provincias que contengan la letra B

In [247]:
conn.execute("""
    SELECT provincia FROM provincias
        WHERE provincia LIKE '%b%';
""")
rs = conn.fetchall()
print(rs)

(('Cantabria',), ('Barcelona',), ('Córdoba',), ('Badajoz',), ('Burgos',), ('Albacete',), ('Baleares',))


In [248]:
df[(
    df['provincia'].str.contains('B') | 
    df['provincia'].str.contains('b')
)].provincia

5     Cantabria
14    Barcelona
24      Córdoba
31      Badajoz
37       Burgos
44     Albacete
49     Baleares
Name: provincia, dtype: object

### Provincias que empiecen por B

In [249]:
conn.execute("""
    SELECT provincia FROM provincias
        WHERE provincia LIKE 'b%';
""")
rs = conn.fetchall()
print(rs)

(('Barcelona',), ('Badajoz',), ('Burgos',), ('Baleares',))


In [250]:
df[ df.provincia.str[0]=='B' ].provincia

14    Barcelona
31      Badajoz
37       Burgos
49     Baleares
Name: provincia, dtype: object

### Provincia con los mismos caracteres que su autonomía

In [251]:
conn.execute("""
    SELECT provincia FROM provincias
        WHERE CHAR_LENGTH(provincia)=CHAR_LENGTH(autonomia);
""")
rs = conn.fetchall()
rs

(('Cantabria',), ('La Rioja',), ('Huesca',), ('Teruel',))

In [252]:
df[ df.provincia.str.len()==df.autonomia.str.len() ].provincia

5     Cantabria
9      La Rioja
11       Huesca
13       Teruel
Name: provincia, dtype: object

### Lista de autonomías

In [253]:
conn.execute("""
    SELECT DISTINCT autonomia FROM provincias;
""")
rs = conn.fetchall()
rs

(('Galicia',),
 ('Principado de Asturias',),
 ('Cantabria',),
 ('País Vasco',),
 ('La Rioja',),
 ('Comunidad Foral de Navarra',),
 ('Aragón',),
 ('Cataluña',),
 ('Comunidad Valenciana',),
 ('Región de Murcia',),
 ('Andalucía',),
 ('Extremadura',),
 ('Comunidad de Madrid',),
 ('Castilla y León',),
 ('Castilla-La Mancha',),
 ('Canarias',),
 ('Islas Baleares',))

In [254]:
df.autonomia.unique()

array(['Galicia', 'Principado de Asturias', 'Cantabria', 'País Vasco',
       'La Rioja', 'Comunidad Foral de Navarra', 'Aragón', 'Cataluña',
       'Comunidad Valenciana', 'Región de Murcia', 'Andalucía',
       'Extremadura', 'Comunidad de Madrid', 'Castilla y León',
       'Castilla-La Mancha', 'Canarias', 'Islas Baleares'], dtype=object)

### Provincias de más de un millón de habitantes

In [255]:
conn.execute("""
    SELECT provincia FROM provincias
        WHERE poblacion>1e6;
""")
rs = conn.fetchall()
for i in rs: print(i[0])

La Coruña
Asturias
Vizcaya
Barcelona
Valencia
Alicante
Murcia
Sevilla
Málaga
Cádiz
Madrid
Santa Cruz de Tenerife
Las Palmas
Baleares


In [256]:
df[ df.poblacion>1e6 ].provincia

0                  La Coruña
4                   Asturias
6                    Vizcaya
14                 Barcelona
19                  Valencia
20                  Alicante
21                    Murcia
23                   Sevilla
28                    Málaga
29                     Cádiz
32                    Madrid
47    Santa Cruz de Tenerife
48                Las Palmas
49                  Baleares
Name: provincia, dtype: object

### Población de España

In [257]:
conn.execute("""
    SELECT SUM(poblacion) FROM provincias;
""")
rs = conn.fetchall()
print(rs[0][0])

46386463


In [258]:
df.poblacion.sum()

46386463

### Densidad de población de España

In [259]:
conn.execute("""
    SELECT SUM(poblacion)/SUM(superficie) FROM provincias;
""")
rs = conn.fetchall()
print(rs[0][0])

91.7274


In [260]:
df.poblacion.sum()/df.superficie.sum()

91.7274169021493

### Habitantes de la provincia de más poblada

In [261]:
conn.execute("""
    SELECT MAX(poblacion) FROM provincias;
""")
rs = conn.fetchall()
rs[0][0]

6466996

In [262]:
df.poblacion.max()

6466996

### Población de Cataluña

In [263]:
conn.execute("""
    SELECT SUM(poblacion) FROM provincias
        WHERE autonomia='Cataluña';
""")
rs = conn.fetchall()
print(rs[0][0])

7522596


In [264]:
df[ df.autonomia=='Cataluña' ].sum().poblacion

7522596

### Número de provincias de cada autonomía

In [265]:
conn.execute("""
    SELECT autonomia,COUNT(*) FROM provincias
        GROUP BY 1;
""")
rs = conn.fetchall()
rs

(('Andalucía', 8),
 ('Aragón', 3),
 ('Canarias', 2),
 ('Cantabria', 1),
 ('Castilla y León', 9),
 ('Castilla-La Mancha', 5),
 ('Cataluña', 4),
 ('Comunidad de Madrid', 1),
 ('Comunidad Foral de Navarra', 1),
 ('Comunidad Valenciana', 3),
 ('Extremadura', 2),
 ('Galicia', 4),
 ('Islas Baleares', 1),
 ('La Rioja', 1),
 ('País Vasco', 3),
 ('Principado de Asturias', 1),
 ('Región de Murcia', 1))

In [266]:
df.groupby('autonomia').count().provincia

autonomia
Andalucía                     8
Aragón                        3
Canarias                      2
Cantabria                     1
Castilla y León               9
Castilla-La Mancha            5
Cataluña                      4
Comunidad Foral de Navarra    1
Comunidad Valenciana          3
Comunidad de Madrid           1
Extremadura                   2
Galicia                       4
Islas Baleares                1
La Rioja                      1
País Vasco                    3
Principado de Asturias        1
Región de Murcia              1
Name: provincia, dtype: int64

### Población de cada autonomía

In [267]:
conn.execute("""
    SELECT autonomia,SUM(poblacion) s FROM provincias
        GROUP BY 1;
""")
rs = conn.fetchall()
rs

(('Andalucía', Decimal('8388107')),
 ('Aragón', Decimal('1308563')),
 ('Canarias', Decimal('2101924')),
 ('Cantabria', Decimal('582206')),
 ('Castilla y León', Decimal('2447519')),
 ('Castilla-La Mancha', Decimal('2041631')),
 ('Cataluña', Decimal('7522596')),
 ('Comunidad de Madrid', Decimal('6466996')),
 ('Comunidad Foral de Navarra', Decimal('640647')),
 ('Comunidad Valenciana', Decimal('4959968')),
 ('Extremadura', Decimal('1087778')),
 ('Galicia', Decimal('2718525')),
 ('Islas Baleares', Decimal('1107220')),
 ('La Rioja', Decimal('315794')),
 ('País Vasco', Decimal('2189534')),
 ('Principado de Asturias', Decimal('1042608')),
 ('Región de Murcia', Decimal('1464847')))

In [268]:
df.groupby('autonomia').sum().poblacion

autonomia
Andalucía                     8388107
Aragón                        1308563
Canarias                      2101924
Cantabria                      582206
Castilla y León               2447519
Castilla-La Mancha            2041631
Cataluña                      7522596
Comunidad Foral de Navarra     640647
Comunidad Valenciana          4959968
Comunidad de Madrid           6466996
Extremadura                   1087778
Galicia                       2718525
Islas Baleares                1107220
La Rioja                       315794
País Vasco                    2189534
Principado de Asturias        1042608
Región de Murcia              1464847
Name: poblacion, dtype: int64

### Provincia más despoblada

In [269]:
conn.execute("""
    SELECT provincia FROM provincias
        WHERE poblacion=(
            SELECT MIN(poblacion)
                FROM provincias
        );
""")
rs = conn.fetchall()
print(rs[0][0])

Soria


In [270]:
df[ df.poblacion.min()==df.poblacion ].provincia.str.split()

41    [Soria]
Name: provincia, dtype: object

### Autonomía más poblada

In [271]:
conn.execute("""
    SELECT autonomia FROM provincias
        GROUP BY 1 HAVING SUM(poblacion)=(
            SELECT MAX(h) FROM (
                SELECT autonomia,SUM(poblacion) h 
                    FROM provincias GROUP BY 1            
            ) c1
        );
""")
rs = conn.fetchall()
rs[0][0]

'Andalucía'

In [272]:
dfa = df.groupby(['autonomia']).sum()
aut = dfa[ dfa.poblacion.max()==dfa.poblacion ].index
aut.str.split()[0][0]

'Andalucía'

### Vamos a añadir Ceuta y Melilla como ciudades autónomas

In [273]:
conn.execute("""
    INSERT INTO provincias (autonomia,provincia) VALUES
        ('Ciudades autónomas','Ceuta'),
        ('Ciudades autónomas','Melilla');        
""")
# sin commit no se registra definitivamente en la base de datos
# aunque sí lo tendremos disponible para consultas en esta sesión

2

In [274]:
#df.loc[50]=['Ciudades autónomas','Ceuta','','']
#df.loc[51]=['Ciudades autónomas','Melilla','','']
df.tail()

,autonomia,provincia,poblacion,superficie
45,Castilla-La Mancha,Cuenca,201071,17140
46,Castilla-La Mancha,Guadalajara,252882,12214
47,Canarias,Santa Cruz de Tenerife,1004124,3381
48,Canarias,Las Palmas,1097800,4065
49,Islas Baleares,Baleares,1107220,4991


In [275]:
df = df.append([
    {'autonomia':'Ciudades autónomas','provincia':'Ceuta'},
    {'autonomia':'Ciudades autónomas','provincia':'Melilla'}
],ignore_index=True)
df.tail()

,autonomia,provincia,poblacion,superficie
47,Canarias,Santa Cruz de Tenerife,1004124.0,3381.0
48,Canarias,Las Palmas,1097800.0,4065.0
49,Islas Baleares,Baleares,1107220.0,4991.0
50,Ciudades autónomas,Ceuta,NaN,NaN
51,Ciudades autónomas,Melilla,NaN,NaN


### ¿Cuántas provincias hay ahora?

In [276]:
conn.execute("""
    SELECT COUNT(*) FROM provincias;
""")
rs = conn.fetchall()
rs[0][0]

52

In [277]:
df.count().provincia

52

### Localiza filas con habitantes nulos

In [278]:
conn.execute("""
    SELECT * FROM provincias
        WHERE poblacion IS NULL;
""")
rs = conn.fetchall()
rs

(('Ciudades autónomas', 'Ceuta', None, None),
 ('Ciudades autónomas', 'Melilla', None, None))

In [279]:
df[ df.poblacion.isna() ]

,autonomia,provincia,poblacion,superficie
50,Ciudades autónomas,Ceuta,NaN,NaN
51,Ciudades autónomas,Melilla,NaN,NaN


### Ceuta tiene 85209 habitantes, actualiza tanto la tabla como el DataFrame

In [280]:
conn.execute("""
    UPDATE provincias
        SET poblacion=85209
        WHERE provincia='Ceuta';
""")

InternalError: (1205, 'Lock wait timeout exceeded; try restarting transaction')

In [ ]:
conn.execute("""
    SELECT * FROM  provincias
        WHERE provincia="Ceuta";
""")
rs = conn.fetchall()
rs

In [283]:
df[ df.provincia=='Ceuta' ].poblacion.value=85209
df.tail()

,autonomia,provincia,poblacion,superficie
47,Canarias,Santa Cruz de Tenerife,1004124.0,3381.0
48,Canarias,Las Palmas,1097800.0,4065.0
49,Islas Baleares,Baleares,1107220.0,4991.0
50,Ciudades autónomas,Ceuta,NaN,NaN
51,Ciudades autónomas,Melilla,NaN,NaN
